# Setup

In [1]:
import os
from pathlib import Path

from PIL import Image
from ultralytics import YOLO

import matplotlib.pyplot as plt
import cv2

In [2]:
root_dir = Path(os.getcwd()).parent

data_dir = root_dir/'data'
processed_data_dir = data_dir/'1_processed'
prediction_dir = data_dir/'2_predictions'
yaml_dir = processed_data_dir/'custom_data.yaml'

runs_dir = root_dir/'runs'

# Training

In [3]:
train_project = 'custom_runs' # custom run directory
data_version = "3.0" # for custom run name

base_model = "yolov8m.pt"
base_model_size = base_model.split('.')[0][-1]

In [4]:
model_start = 'pretrained'
if model_start == 'pretrained':
    pretrained_flag = True
else:
    pretrained_flag = False

In [5]:
cont = True

In [6]:
custom_project_dir = root_dir / train_project
if cont:
    runs_list = os.listdir(custom_project_dir)
    last_run = runs_list[-1]
else:
    # create custom run directory
    if not os.path.exists(custom_project_dir):
        os.makedirs(custom_project_dir)

    # get the list of run
    runs_list = os.listdir(custom_project_dir)

    # create custom run name
    if not runs_list:
        train_name = f"0_{data_version}_{model_start}_{base_model_size}"
    else:
        last_run = max([int(order.split('_')[0]) for order in runs_list])
        new_run = last_run + 1
        train_name = f"{new_run}_{data_version}_{model_start}_{base_model_size}"

In [7]:
if cont:
    # Load a model
    model = YOLO(custom_project_dir / last_run / 'weights' / 'last.pt')  # load a partially trained model

    # Resume training
    results = model.train(resume=True)
else:
    # Load a model
    model = YOLO(
        base_model
        )

    # train the model
    results = model.train(
        data=yaml_dir,
        epochs=300,
        batch=12,
        patience=10,
        project=custom_project_dir,
        name=train_name,
        pretrained=pretrained_flag
    )

New https://pypi.org/project/ultralytics/8.0.205 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.200  Python-3.9.18 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=d:\0_amri_local\14_pupr_roaddamagedetection\yolov8_custom\custom_runs\2_3.0_pretrained_m\weights\last.pt, data=d:\0_amri_local\14_pupr_roaddamagedetection\yolov8_custom\data\1_processed\custom_data.yaml, epochs=300, patience=10, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=d:\0_amri_local\14_pupr_roaddamagedetection\yolov8_custom\custom_runs, name=2_3.0_pretrained_m, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, co

KeyboardInterrupt: 

# Inferencing

## Image

In [11]:
# Load a pretrained YOLOv8n model
best_model = YOLO(custom_project_dir / train_name / "weights" / "best.pt")

In [35]:
pred_data = [str(prediction_dir / "original" / file) for file in os.listdir(prediction_dir / "original")]

pred_location = prediction_dir / train_name
if not os.path.exists(pred_location):
    os.makedirs(pred_location)

In [30]:
# Run inference on an image
results = best_model(
    source = pred_data
)  # list of 1 Results object


0: 640x640 1 retak_buaya, 1: 640x640 2 retak_memanjangs, 2: 640x640 3 retak_memanjangs, 3: 640x640 (no detections), 4: 640x640 1 retak_buaya, 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 3 retak_buayas, 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 4 retak_memanjangs, 14: 640x640 2 retak_memanjangs, 15: 640x640 (no detections), 16: 640x640 (no detections), 290.2ms
Speed: 4.5ms preprocess, 17.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


In [34]:
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # im.show()  # show image
    im.save(pred_location / ('pred_'+Path(r.path).name))

## Video

In [ ]:
video_path = r"D:\0_amri_local\14_pupr_roaddamagedetection\data\PUPR\Banten\21017 R1.mp4"
video_path_out = r"D:\0_amri_local\14_pupr_roaddamagedetection\yolov8_custom\notebooks\out.mp4"

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

In [ ]:
threshold = 0.5

while ret:

    results = best_model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()

# Placeholder